In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-rqtebh12
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-rqtebh12


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 이상한 문자 제거
def Strange(df):
    df['text'] = df['text'].map(lambda x : x.replace('Â', ''))
    return df
# 챕터명 제거
def Chapter(df):
    chapter = ['CHAPTER I' , 'CHAPTER V', 'CHAPTER X' ]
    for i in chapter:
        df = df[~df['text'].str.contains(i, na=False, case=False)]
    return df
# CHORUS 제거
def Chorus(df):
    df = df[~df['text'].str.contains('CHORUS.', na=False, case=False)]
    return df
# 괄호 제거
def gwalho1(string):
    pattern = '\[[^)]*\]'
    return ' '.join(re.sub(pattern=pattern, repl='', string= string).split())
def gwalho2(string):
    pattern2 = '\([^)]*\)'
    return ' '.join(re.sub(pattern=pattern2, repl='', string= string).split())
def gwalho3(string):
    pattern2 = '\{[^)]*\}'
    return ' '.join(re.sub(pattern=pattern2, repl='', string= string).split())
# -제거
# def hyphen(string):
#     pattern2 = '\-[^)]*\-'
#     return ' '.join(re.sub(pattern=pattern2, repl='', string= string).split())
def clean_text(inputString):
  text_rmv = re.sub('[-=+#/\:^@*\※~ㆍ{}』·]', ' ', inputString)
  return text_rmv

In [ ]:
def PreProcessing(df):
    #df = pd.DataFrame([x.strip() for x in texts.split('\n\r\n')],columns = ['text'])
    df = Strange(df)
    df = Chapter(df)
    df = Chorus(df)
    df['text'] = df['text'].map(lambda x: gwalho1(x))
    df['text'] = df['text'].map(lambda x: gwalho2(x))
    df['text'] = df['text'].map(lambda x: gwalho3(x))
    # df['text'] = df['text'].map(lambda x: hyphen(x))
    df['text'] = df['text'].map(lambda x: clean_text(x))
    df = df[df['text'] != '']
    return df

In [ ]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
file_names = gutenberg.fileids() #파일 제목을 읽어온다.
print(file_names)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [ ]:
# 동화 데이터를 문장 단위로 쪼갬
texts = []
for file_name in file_names:
    doc_alice = gutenberg.open(file_name).read()
    texts.extend([ x for x in doc_alice.split('\n\n') if x != ''])

In [ ]:
df1 = pd.DataFrame(texts,columns = ['text'])

In [ ]:
# 글자수 350 초과하는 문장은 제거 (1024토큰이 넘으면 작동안함)
df1 = df1[df1['text'].apply(lambda x: len(x.split(' ')) < 350)]

# 목차 이름 제거
df1 = df1[~df1['text'].str.contains('CHAPTER', na=False, case=False)]
df1 = df1[~df1['text'].str.contains('VOLUME', na=False, case=False)]

In [ ]:
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1

,text
0,[Emma by Jane Austen 1816]
1,"\nEmma Woodhouse, handsome, clever, and rich, ..."
2,She was the youngest of the two daughters of a...
3,Sixteen years had Miss Taylor been in Mr. Wood...
4,"The real evils, indeed, of Emma's situation we..."
...,...
39383,} Good-Bye My Fancy!
39384,"Good-bye my Fancy!\nFarewell dear mate, dear l..."
39385,Now for my last--let me look back a moment;\nT...
39386,"Long have we lived, joy'd, caress'd together;\..."


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataSet/fairytale_texts.csv')

In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df = pd.concat([df1,df], axis = 0)
df

,text
0,[Emma by Jane Austen 1816]
1,"\nEmma Woodhouse, handsome, clever, and rich, ..."
2,She was the youngest of the two daughters of a...
3,Sixteen years had Miss Taylor been in Mr. Wood...
4,"The real evils, indeed, of Emma's situation we..."
...,...
52219,"He embraced Sasha and Lida, who hung upon his ..."
52220,"""Grandpapa sends his love. . . . Uncle Fyodor ..."
52221,Then he sat on the verandah and saw his wife w...
52222,And he thought:


In [ ]:
df = PreProcessing(df)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,text
0,"Emma Woodhouse, handsome, clever, and rich, wi..."
1,She was the youngest of the two daughters of a...
2,Sixteen years had Miss Taylor been in Mr. Wood...
3,"The real evils, indeed, of Emma's situation we..."
4,Sorrow came a gentle sorrow but not at all in ...
...,...
91473,"He embraced Sasha and Lida, who hung upon his ..."
91474,"""Grandpapa sends his love. . . . Uncle Fyodor ..."
91475,Then he sat on the verandah and saw his wife w...
91476,And he thought


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataSet/fairytale123.csv')

In [ ]:
# 데이터셋 인코딩
class Create_data(Dataset):  
    def __init__(self, control_code, truncate=True, gpt2_type="gpt2", max_length=1024, padding=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.text = []

        for row in df['text']:
            self.text.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))

        if truncate:
            self.text = self.text[:20000]
        self.text_count = len(self.text)
        
    def __len__(self):
        return self.text_count

    def __getitem__(self, item):
        print(self.text[item])
        return self.text[item]
    
dataset = Create_data(df['text'], truncate=True, gpt2_type="gpt2") 

In [ ]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


20000it [17:14, 19.34it/s]


Training epoch 1
tensor(0.5137, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [16:50, 19.79it/s]


...

20000it [16:50, 19.79it/s]


Training epoch 3
tensor(1.3431, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [16:44, 19.90it/s]


Training epoch 4
tensor(0.2833, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [16:48, 19.84it/s]


In [ ]:
# 토크나이저 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
# 모델 저장
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/gpt_model.pt')

In [ ]:
# 토크나이저 불러오기
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)

AttributeError: ignored

In [ ]:
# 모델 불러오기
model = torch.load('/content/drive/MyDrive/Colab Notebooks/gpt_model2.pt')

ModuleNotFoundError: ignored

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=50, #maximum number of words
    top_p=0.92,
    temperature=0.8,
):

    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [ ]:
import re
import random
import torch.backends.cudnn as cudnn
#from googletrans import Translator
#from hanspell import spell_checker

def generate_text(text):
    res = generate(model.to('cpu'), tokenizer, text, entry_count=1)[0]
    to_remove = res.split('.')[-1]
    res = res.replace(to_remove,'')
    print()
    return res

# 시드 고정
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

# <|endoftext|> 제거
# pattern = '\<[^)]*\>'
# a = ' '.join(re.sub(pattern=pattern, repl='', string= res).split())
# print(a)

# # 한국어로 번역
# translator = Translator()
# result = translator.translate(res, dest="ko")

# # 문법 교정
# spelled_sent = spell_checker.check(result.text)
# hanspell_sent = spelled_sent.checked
# print(hanspell_sent)

In [ ]:
print(generate_text('The next day, the prince looks for the owner of the shoe.'))

100%|██████████| 1/1 [00:15<00:00, 15.18s/it]


The next day, the prince looks for the owner of the shoe. His last name is Bernardo.

Bernardo is usually dressed in a gold suit with a black button down.

The prince walks to the table and asks where the owner is.


In [ ]:
generate_text('Cinderella lives with her stepmother and two stesisters')

100%|██████████| 1/1 [00:10<00:00, 10.71s/it]

'Cinderella lives with her stepmother and two stesisters in a house in the far north of town. When the sisters arrive, Cinderella has found her mother and is in fear for her life.'